In [131]:
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.simplefilter('ignore')

In [132]:
country_virus=pd.read_csv("country_by_date.csv")
country_age = pd.read_csv("age_ratio.csv")
country_age = country_age.rename(columns={'Entity': 'Country_Region'})
country_age = country_age.loc[(country_age['Year'] == 2017)]
country_age = country_age[['Country_Region','Age dependency ratio, old (% of working-age population) (% of working-age population)']]

In [133]:
country_virus

,Unnamed: 0,Country_Region,Date,ConfirmedCases
0,0,Afghanistan,2020-01-22,0.0
1,5456,Afghanistan,2020-02-12,0.0
2,5458,Afghanistan,2020-02-10,0.0
3,5459,Afghanistan,2020-02-09,0.0
4,5460,Afghanistan,2020-02-08,0.0
...,...,...,...,...
13315,5415,Zimbabwe,2020-03-05,0.0
13316,5416,Zimbabwe,2020-03-04,0.0
13317,5427,Zimbabwe,2020-02-15,0.0
13318,5429,Zimbabwe,2020-02-17,0.0


In [134]:
#create a new table to get the country demographics data
countries = country_virus['Country_Region'].unique()
countries = pd.DataFrame (countries)
countries = countries.rename(columns={0: 'Country_Region'})

country_age = country_age.rename(columns={'Age dependency ratio, old (% of working-age population) (% of working-age population)':'Age Dependency Ratio (65+/Working Age)'})
country_age

,Country_Region,Age Dependency Ratio (65+/Working Age)
57,Afghanistan,4.763323
115,Albania,18.997890
173,Algeria,9.628361
231,Angola,4.734531
289,Antigua and Barbuda,10.021966
...,...,...
13792,Zambia,4.701905
13850,Zimbabwe,5.041526
13851,Monaco,44.200000
13852,Dominica,10.910000


In [135]:
physician = pd.read_csv("physician_per_capita_edit.csv")
physician = physician[['Country Name','Most_Recent']]
physician = physician.rename(columns={'Country Name': 'Country_Region'})
physician = physician.rename(columns={'Most_Recent': 'Physician_per_thousand'})

In [136]:
other_demo = pd.read_csv("other_demo_info_edit.csv")

In [137]:
# import datetime
# ts = pd.Timestamp(year = 2011,  month = 11, day = 21,  
#                   hour = 10, second = 49, tz = 'US/Pacific')
other_demo = other_demo[['country','hospibed','pop']]
#other_demo['quarantine time'] = other_demo.groupby(['Country_Region'])['quarantine'].max()
other_demo = other_demo.rename(columns={'country': 'Country_Region'})


In [138]:
other_demo

,Country_Region,hospibed,pop
0,Afghanistan,0.5,38928346.0
1,Albania,2.9,2877797.0
2,Algeria,1.9,43851044.0
3,Andorra,2.5,77265.0
4,Antigua and Barbuda,3.8,97929.0
...,...,...,...
175,Venezuela,0.8,28435940.0
176,Vietnam,2.6,97338579.0
177,Zambia,2.0,18383955.0
178,Zimbabwe,1.7,14862924.0


In [139]:
lockdown_dates=pd.read_csv("countryLockdowndates_edit.csv")
lock_down =  lockdown_dates[['Country/Region','Date']]
lock_down = lock_down.rename(columns={'Country/Region': 'Country_Region','Date':'Lockdown Date'})

In [140]:
country_by_date=pd.read_csv("join-table/country_by_date.csv")
country_demographics_date = country_by_date.merge(country_age, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(physician, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(other_demo, on=['Country_Region'], how='left') 
country_demographics_date = country_demographics_date.merge(lock_down, on=['Country_Region'], how='left')
country_demographics_date

,Unnamed: 0,i,Country_Region,Date,ConfirmedCases,Population Density (per sq. km),HDI,Testing By Date,Total Testing,Age Dependency Ratio (65+/Working Age),Physician_per_thousand,hospibed,pop,Lockdown Date
0,0,0,Afghanistan,2020-01-22,0,56.937760,0.498,NaN,NaN,4.763323,0.3039,0.5,38928346.0,24/03/2020
1,1,5476,Afghanistan,2020-01-23,0,56.937760,0.498,NaN,NaN,4.763323,0.3039,0.5,38928346.0,24/03/2020
2,2,5475,Afghanistan,2020-01-24,0,56.937760,0.498,NaN,NaN,4.763323,0.3039,0.5,38928346.0,24/03/2020
3,3,5474,Afghanistan,2020-01-25,0,56.937760,0.498,NaN,NaN,4.763323,0.3039,0.5,38928346.0,24/03/2020
4,4,5473,Afghanistan,2020-01-26,0,56.937760,0.498,NaN,NaN,4.763323,0.3039,0.5,38928346.0,24/03/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13315,13315,9279,Zimbabwe,2020-03-31,8,37.324591,0.535,NaN,NaN,5.041526,0.2020,1.7,14862924.0,27/03/2020
13316,13316,9277,Zimbabwe,2020-04-01,8,37.324591,0.535,NaN,NaN,5.041526,0.2020,1.7,14862924.0,27/03/2020
13317,13317,9372,Zimbabwe,2020-04-02,9,37.324591,0.535,NaN,NaN,5.041526,0.2020,1.7,14862924.0,27/03/2020
13318,13318,9443,Zimbabwe,2020-04-03,9,37.324591,0.535,NaN,NaN,5.041526,0.2020,1.7,14862924.0,27/03/2020


In [141]:
country_demographics_date.to_csv("demographics/country_demographics_by_date.csv")

In [142]:
country_demographics = country_demographics_date[['Country_Region','Population Density (per sq. km)','HDI',
                                                  'Total Testing','hospibed','pop','Age Dependency Ratio (65+/Working Age)',
                                                 'Physician_per_thousand','Lockdown Date']]

In [143]:
country_demographics = country_demographics.drop_duplicates()

In [144]:
country_demographics.to_csv("demographics/country_demographics.csv")